## Notes 
<b> 2/18/2022 <b>
* For SAR, only projects that have empty phase completion dates are included. 
* Projects with NO allocation amount won't appear in SAR. 
* Wants to compare cols from previous SAR with current SAR b/c she has to bold any changes in black for CTC.
* For PAP, problem is prior col is all lumped together for projects before 2020? Highlands sheet does not differentiate between the years. Wants each year to be separated out.
    
<b> To do </b>
* Make sure all the values in date columns are replaced properly. 
* Make sure grant recipients don't have multiple variations of the same recipient.
* Change col names to match the new SAR report sent over 2/15.
* SAR: allocation amount col isn't from highlands sheet, it's taken by adding GGRF Alloc + PTA-SB1. Add them back in, add the column...

In [1]:
import numpy as np
import pandas as pd
import TIRCP_functions
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/tircp/"
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:.2f}".format


/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
#Load in Crosswalks 
FILE_NAME3 = "Allocation_PPNO_Crosswalk.csv"
allocation_ppno_crosswalk = pd.read_csv(f"{GCS_FILE_PATH}{FILE_NAME3}")
    
#Allocation PPNO Crosswalk
FILE_NAME4 = "Projects_PPNO.xlsx"
project_ppno_crosswalk = pd.read_excel(f"{GCS_FILE_PATH}{FILE_NAME4}")


### Loading in Scripts

In [3]:
test_SAR = TIRCP_functions.semi_annual_report()

In [4]:
test_program = TIRCP_functions.program_allocation_plan()

In [5]:
alloc_test = TIRCP_functions.allocation()

In [6]:
tableau_test = TIRCP_functions.tableau()

In [7]:
project_test = TIRCP_functions.project()

### Testing

In [8]:
def tableau():
    #Keeping only the columns we want
    df = project_test #CHANGE THIS BACK TO JUST project()
    
    #Getting percentages & filling in with 0
    df['Expended_Percent'] = df['Expended_Amt_project_sheet']/df['Allocated_Amount']
    df['Allocated_Percent'] = df['Allocated_Amount']/df['TIRCP_project_sheet']
    df[['Expended_Percent','Allocated_Percent']] = df[['Expended_Percent','Allocated_Percent']].fillna(value=0)
    
    #Categorizing expended percentage into bins
    def expended_percent(row):
            if row.Expended_Percent == 0:
                return "No expenditure recorded"
            elif ((row.Expended_Percent > 0) and (row.Expended_Percent < .50)):
                return "1-50"
            elif row.Expended_Percent < 0.71:
                return "51-70"
            else:
                return "71-100"
    df["Expended_Percent_Group"] = df.apply(lambda x: expended_percent(x), axis=1)
    
    # Categorize years and expended_percent_group into bins
    def progress(df):   
        if (df['Award_Year'] == 2015) and (df['Expended_Percent_Group'] == "1-50"):
            return 'Behind'
        elif (df['Award_Year'] == 2015) and (df['Expended_Percent_Group'] == "51-70"):
            return 'On Track'
        elif (df['Award_Year'] == 2015) and (df['Expended_Percent_Group'] == "71-100"):
            return 'On Track'
        elif (df['Award_Year'] == 2016) and (df['Expended_Percent_Group'] == "1-50"):
            return 'Behind'
        elif (df['Award_Year'] == 2016) and (df['Expended_Percent_Group'] == "71-100"):
            return 'On Track'
        elif (df['Award_Year'] == 2016) and (df['Expended_Percent_Group'] == "51-70"):
            return 'On Track'
        elif (df['Award_Year'] == 2018) and (df['Expended_Percent_Group'] == "1-50"):
            return 'On Track'
        elif (df['Award_Year'] == 2018) and (df['Expended_Percent_Group'] == "51-70"):
            return 'Ahead'
        elif (df['Award_Year'] == 2018) and (df['Expended_Percent_Group'] == "71-100"):
            return 'Ahead'
        elif (df['Award_Year'] == 2020) and (df['Expended_Percent_Group'] == "1-50"):
            return 'On Track'
        elif (df['Award_Year'] == 2020) and (df['Expended_Percent_Group'] == "51-70"):
            return 'Ahead'
        elif (df['Award_Year'] == 2020) and (df['Expended_Percent_Group'] == "71-100"):
            return 'Ahead'
        else: 
            return "No Expenditures"

    df['Progress'] = df.apply(progress, axis = 1)
    
    #Renaming districts
    df['District'] = (df['District'].replace({7:'District 7: Los Angeles',
                                            4:'District 4: Bay Area / Oakland',
                                            'VAR':'Various',
                                            10:'District 10: Stockton',
                                            11:'District 11: San Diego',
                                            3:'District 3: Marysville / Sacramento',
                                            12: 'District 12: Orange County',
                                            8: 'District 8: San Bernardino / Riverside',
                                            5:'District 5: San Luis Obispo / Santa Barbara',
                                            6:'District 6: Fresno / Bakersfield',
                                            1:'District 1: Eureka'
                                                 }))
    #Renaming counties
    df['County'] = (df['County'].replace({'LA': 'Los Angeles', 
                                          'VAR': 'Various', 
                                          'MON': 'Mono', 
                                          'ORA':'Orange', 
                                          'SAC':'Sacramento', 
                                          'SD':'San Diego', 
                                          'SF': 'San Francisco', 
                                          'SJ':'San Joaquin', 
                                          'SJ ': 'San Joaquin', 
                                          'FRE':"Fresno",
                                           'SBD':'San Bernandino', 
                                          'SCL':'Santa Clara', 
                                          'ALA':'Alameda', 
                                          'SM':'San Mateo', 
                                          'SB': 'San Barbara', 
                                          'SON, MRN': 'Various', 

                                                 }))
    
    #Which projects are large,small, medium
    p75 = df.TIRCP_project_sheet.quantile(0.75).astype(float)
    p25 = df.TIRCP_project_sheet.quantile(0.25).astype(float)
    p50 = df.TIRCP_project_sheet.quantile(0.50).astype(float)
    
    def project_size (row):
        if ((row.TIRCP_project_sheet > 0) and (row.TIRCP_project_sheet < p25)):
            return "Small"
        elif ((row.TIRCP_project_sheet > p25) and (row.TIRCP_project_sheet < p75)):
            return "Medium"
        elif ((row.TIRCP_project_sheet > p50) and (row.TIRCP_project_sheet > p75 )):
            return "Large"
        else:
            return "$0 recorded for TIRCP"
        
    df["Project_Category"] = df.apply(lambda x: project_size(x), axis=1)
     #Rename cols to the right names
    df = (df.rename(columns = {'Expended_Amt_project_sheet':'Expended_Amount', 
                                                'TIRCP_project_sheet': "TIRCP_Amount"}
                  ))
    ### GCS ###
    with pd.ExcelWriter(f"{GCS_FILE_PATH}Tableau_Sheet.xlsx") as writer:
        df.to_excel(writer, sheet_name="Data", index=False)
    return df

    return df 

In [9]:
tableau_test = tableau()

In [10]:
tableau_test[['Project_Title','District','County']]

,Project_Title,District,County
0,Regional Transit Interconnectivity & Environme...,District 7: Los Angeles,Los Angeles
1,Travel Time Reduction Project,District 4: Bay Area / Oakland,Various
2,Willowbrook/Rosa Parks Station & Blue Line Lig...,District 7: Los Angeles,Los Angeles
3,Pacific Surfliner Transit Transfer Program,Various,Various
4,Monterey Bay Operations and Maintenance Facili...,District 5: San Luis Obispo / Santa Barbara,Mono
5,Bravo! Route 560 Rapid Buses,District 12: Orange County,Orange
6,Sacramento Regional Transit's Refurbishment of...,District 3: Marysville / Sacramento,Sacramento
7,South Bay Bus Rapid Transit,District 11: San Diego,San Diego
8,San Diego Metropolitan Transit System Trolley ...,District 11: San Diego,San Diego
9,SFMTA Light Rail Vehicle Fleet Expansion,District 4: Bay Area / Oakland,San Francisco


### Once I get the final data, compare the PPNO # across the 2 sheets 

In [11]:
PPNO_project = set(project_test.PPNO.unique().tolist())
PPNO_allocation = set(alloc_test.PPNO.unique().tolist())

In [12]:
PPNO_project - PPNO_allocation #checking for differences

set()